In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
pwd

In [ ]:
df=pd.read_csv("C:\\Users\\USER\\Documents\\PapersforSelfDevelopment\\PaperSD1\\DataSet.csv")
#df=pd.read_csv("C:\\Users\\USER\\Documents\\CleanedDatasetUsed.csv")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.corr()
correlations = df.corr()
correlations = pd.DataFrame(correlations)
correlations.to_csv('correlationsoftheCSTSFSdata.csv')

In [ ]:
stat= df.describe()
stat.to_csv('DescriptionStat.csv')

In [ ]:
df.corr()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
X = df.drop('TS', axis=1)
y = df['TS']

In [ ]:
#Check the shape of the data at this point
df.shape

In [ ]:
#information on the data
df.info()

In [ ]:
#statistics about the data
df.describe()

In [ ]:
#check for null values
df.isnull().sum()

In [ ]:
#EDA commences here
for column in df.columns:
    plt.figure(figsize=(13,4))
    sns.boxplot(x=column,data=df);

In [ ]:
#produce a heatmap of the above result
plt.figure(figsize=(11,9))
sns.heatmap(data=df.corr(),annot=True)
plt.savefig('correlation coefficient.tif', dpi=600, bbox_inches='tight')

In [ ]:
#conduct a univariate analysis

#make a histogram
import matplotlib.pyplot as plt

# Create histogram plots
axes = df.hist(figsize=(11, 9))

# Save the figure
plt.savefig('Distribution.tif', dpi=600, bbox_inches='tight')

plt.close()

In [ ]:
#attempt to make data normally distributed, split data first in train-val-test

X=df.drop(['CS'],axis=1) #X is "remove the variable in '' "
y=df['CS']

In [ ]:
from sklearn.model_selection import train_test_split
X_model_train, X_test, y_model_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
y_model_train.to_csv("TrainingY_70-30.csv")

In [ ]:
X_test

In [ ]:
y_model_train.to_csv('yfortrain30percent.csv')

In [ ]:
pwd

In [ ]:
Trainingset=pd.read_csv("C:\\Users\\USER\\Documents\\PapersforSelfDevelopment\\PaperSD1\\CSandFSandTS\\TrainingSet_70-30-CS.csv")
Testset=pd.read_csv("C:\\Users\\USER\\Documents\\PapersforSelfDevelopment\\PaperSD1\\CSandFSandTS\\TestingSet_70-30-CS.csv")

In [ ]:
TrainingsetCB=pd.read_csv("C:\\Users\\USER\\Documents\\PapersforSelfDevelopment\\PaperSD1\\CSandFSandTS\\TrainingSetfor_70-30.csv")
TestsetCB=pd.read_csv("C:\\Users\\USER\\Documents\\PapersforSelfDevelopment\\PaperSD1\\CSandFSandTS\\TestingSetfor_70-30.csv")

In [ ]:
#conduct a univariate analysis
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 12})
#make a histogram
import matplotlib.pyplot as plt

# Create histogram plots
axes = TrainingsetCB.hist(figsize=(11, 9))

# Save the figure
plt.savefig('TrainingsetCB.tif', dpi=600, bbox_inches='tight')

plt.close()

In [ ]:
#conduct a univariate analysis
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 12})
#make a histogram
import matplotlib.pyplot as plt

# Create histogram plots
axes = TestsetCB.hist(figsize=(11, 9))

# Save the figure
plt.savefig('TestsetCB.tif', dpi=600, bbox_inches='tight')

plt.close()

In [ ]:
Trainingset.describe()

In [ ]:
Testset.describe()

In [ ]:
from autogluon.tabular import TabularPredictor


train_data = Trainingset
test_data = Testset

predictor = TabularPredictor(label='CS').fit(train_data=train_data)
predictions = predictor.predict(test_data.drop(columns='CS'))

In [ ]:
predictor.evaluate(train_data, silent=True)

In [ ]:
predictor.evaluate(test_data, silent=True)

In [ ]:
predictor.leaderboard(train_data)

In [ ]:
FSResults=predictor.leaderboard(train_data, extra_metrics=['r2', 'mean_squared_error', 'mse', 'root_mean_squared_error', 'mean_absolute_error', 'median_absolute_error', 
                                                'mean_absolute_percentage_error', 'symmetric_mean_absolute_percentage_error', 'spearmanr', 'pearsonr'])
FSResults.to_csv('FSResults.csv')

In [ ]:
FSTestResults=predictor.leaderboard(test_data, extra_metrics=['r2', 'mean_squared_error', 'mse', 'root_mean_squared_error', 'mean_absolute_error', 'median_absolute_error', 
                                                'mean_absolute_percentage_error', 'symmetric_mean_absolute_percentage_error', 'spearmanr', 'pearsonr'])
FSTestResults.to_csv('FSTestResults.csv')

In [ ]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import sklearn
import shap
import time

shap.initjs()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class AutogluonWrapper:
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names
    
    def predict(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        return self.ag_model.predict(X)

In [ ]:
X_train = train_data.drop(columns=['CS'])
X_test = test_data.drop(columns=['CS'])
y_train = train_data['CS']
y_test = test_data['CS']

X_train_summary = shap.kmeans(X_train, 10)
print("Baseline feature-values: \n", X_train_summary)
def print_accuracy(f):
    print("Root mean squared test error = {0}".format(np.sqrt(np.mean((f(X_test) - y_test)**2))))
    time.sleep(0.5) # to let the print get out before any progress bars
feature_names = X_train.columns

In [ ]:
ag_wrapper = AutogluonWrapper(predictor, feature_names)

print_accuracy(ag_wrapper.predict)

explainer = shap.KernelExplainer(ag_wrapper.predict, X_train_summary)

NSHAP_SAMPLES = 1232  # how many samples to use to approximate each Shapely value, larger values will be slower
N_VAL = 528
# how many datapoints from validation data should we interpret predictions for, larger values will be slower

In [ ]:
ROW_INDEX = 0  # index of an example datapoint
single_datapoint = X_train.iloc[[ROW_INDEX]]
single_prediction = ag_wrapper.predict(single_datapoint)

shap_values_single = explainer.shap_values(single_datapoint, nsamples=NSHAP_SAMPLES)
shap.force_plot(explainer.expected_value, shap_values_single, X_train.iloc[ROW_INDEX,:])

In [ ]:
shap_values = explainer.shap_values(X_test.iloc[0:N_VAL,:], nsamples=NSHAP_SAMPLES)
shap.force_plot(explainer.expected_value, shap_values, X_test.iloc[0:N_VAL,:])

In [ ]:
plt.figure()
shap.summary_plot(shap_values, X_test.iloc[0:N_VAL,:], show=False)
plt.savefig("SummaryPlotCS.tif", dpi=600, bbox_inches='tight')

In [ ]:
plt.figure()
shap.dependence_plot("ER", shap_values, X_test.iloc[0:N_VAL,:], show=False)
plt.savefig('ERdepenceplotforCS.tif', dpi=600, bbox_inches='tight')

In [ ]:
plt.figure()
shap.dependence_plot("UPV", shap_values, X_test.iloc[0:N_VAL,:], show=False)
plt.savefig('UPVdepenceforCS.tif', dpi=600, bbox_inches='tight')

In [ ]:
test_data['CS'] = y_test  # add labels to validation DataFrame
predictor.feature_importance(test_data)

In [ ]:
import shap
import autogluon.tabular

# Load your trained AutoGluon model and data
predictor = autogluon.tabular.TabularPredictor.load("AutoGluonModel")
X_test = test_data.drop(columns=["FS"])

# Use SHAP to explain predictions
explainer = shap.Explainer(predictor.model)
shap_values = explainer(X_test)

# Visualize SHAP values
shap.summary_plot(shap_values, X_test)

In [ ]:
predictor.model_best

In [ ]:
predictor.leaderboard(test_data)

In [ ]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'])

In [ ]:
specific_model = 'KNeighborsDist'  # Replace with your model's name

# Get predictions from the specified model
model_predictions = predictor.predict(train_data, model=specific_model)
predictor.evaluate(train_data, model=specific_model, silent=True)

In [ ]:
import numpy as np
# Assuming 'predictor' is your trained AutoGluon predictor and 'test_data' is your test dataset
# Evaluate the model to get predictions
predictions = predictor.predict(train_data.drop(columns=['FS']), model=specific_model)  # Drop the target column for predictions
actuals = train_data['FS'].values  # Get the actual values from the target column
# Calculate a20-index
def calculate_a20_index(actual, predicted):
    within_20_percent = np.abs((actual - predicted) / actual) <= 0.20
    m20 = np.sum(within_20_percent)
    M = len(actual)
    return (m20 / M) * 100
# Calculate Mean Absolute Percentage Error (MAPE)
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100
# Calculate Willmott’s Index (WI)
def calculate_wi(actual, predicted):
    numerator = np.sum((actual - predicted) ** 2)
    denominator = np.sum((np.abs(predicted - np.mean(actual)) + np.abs(actual - np.mean(predicted))) ** 2)
    return 1 - (numerator / denominator)
# Calculate Nash-Sutcliffe Efficiency (NSE)
def calculate_nse(actual, predicted):
    return 1 - (np.sum((actual - predicted) ** 2) / np.sum((actual - np.mean(actual)) ** 2))
# Compute MAPE
mape = calculate_mape(actuals, predictions)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
# Compute a20-index
a20_index = calculate_a20_index(actuals, predictions)
print(f'a20-index: {a20_index:.4f}%')
# Compute Willmott’s Index (WI)
wi = calculate_wi(actuals, predictions)
print(f'Willmott’s Index (WI): {wi:.4f}')
# Compute Nash-Sutcliffe Efficiency (NSE)
nse = calculate_nse(actuals, predictions)
print(f'Nash-Sutcliffe Efficiency (NSE): {nse:.4f}')


In [ ]:
import pandas as pd

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['FS'])

# Optionally, include the actual values and any identifier columns
actuals = train_data['FS'].values
predictions_df['Actual_FS'] = actuals
predictions_df['ER'] = test_data['ER']  # If you have an identifier column in your test data

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictioKNeighborsDist-FS-Train.csv', index=False)

print('Predictions saved to predictions.csv')

In [ ]:
specific_model = 'KNeighborsDist'  # Replace with your model's name

# Get predictions from the specified model
model_predictions = predictor.predict(test_data, model=specific_model)
predictor.evaluate(test_data, model=specific_model, silent=True)

In [ ]:
predictor.feature_importance(test_data, model=specific_model)

In [ ]:
import numpy as np

# Assuming 'predictor' is your trained AutoGluon predictor and 'test_data' is your test dataset
# Evaluate the model to get predictions
predictions = predictor.predict(test_data.drop(columns=['FS']), model=specific_model)  # Drop the target column for predictions
actuals = test_data['FS'].values  # Get the actual values from the target column

# Calculate a20-index
def calculate_a20_index(actual, predicted):
    within_20_percent = np.abs((actual - predicted) / actual) <= 0.20
    m20 = np.sum(within_20_percent)
    M = len(actual)
    return (m20 / M) * 100

# Calculate Mean Absolute Percentage Error (MAPE)
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

# Calculate Willmott’s Index (WI)
def calculate_wi(actual, predicted):
    numerator = np.sum((actual - predicted) ** 2)
    denominator = np.sum((np.abs(predicted - np.mean(actual)) + np.abs(actual - np.mean(predicted))) ** 2)
    return 1 - (numerator / denominator)

# Calculate Nash-Sutcliffe Efficiency (NSE)
def calculate_nse(actual, predicted):
    return 1 - (np.sum((actual - predicted) ** 2) / np.sum((actual - np.mean(actual)) ** 2))

# Compute MAPE
mape = calculate_mape(actuals, predictions)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')

# Compute a20-index
a20_index = calculate_a20_index(actuals, predictions)
print(f'a20-index: {a20_index:.4f}%')

# Compute Willmott’s Index (WI)
wi = calculate_wi(actuals, predictions)
print(f'Willmott’s Index (WI): {wi:.4f}')

# Compute Nash-Sutcliffe Efficiency (NSE)
nse = calculate_nse(actuals, predictions)
print(f'Nash-Sutcliffe Efficiency (NSE): {nse:.4f}')


In [ ]:
import pandas as pd

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['FS'])

# Optionally, include the actual values and any identifier columns
actuals = test_data['FS'].values
predictions_df['Actual_FS'] = actuals
predictions_df['ER'] = test_data['ER']  # If you have an identifier column in your test data

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictionsKNeighborsDist-FS-Test.csv', index=False)

print('Predictions saved to predictions.csv')

In [ ]:
#Here’s how to specify a particular model to use for prediction instead of AutoGluon’s default model-choice:
i = 5  # index of model to use
model_to_use = predictor.model_names()[i]
model_pred = predictor.predict(test_data, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

In [ ]:
all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

In [ ]:
predictor_information

In [ ]:
predictor.evaluate(test_data)

In [ ]:
We can easily access various information about the trained predictor or a particular model:

In [ ]:
import pandas as pd

# Assuming 'predictor' is your trained AutoGluon predictor and 'train_data' is your train dataset
# Get the predictions
predictions = predictor.predict(train_data.drop(columns=['UCS']))  # Drop the target column for predictions

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['UCS'])

# Optionally, include the actual values and any identifier columns
actuals = train_data['UCS'].values
predictions_df['Actual_UCS'] = actuals
predictions_df['Curing_days'] = train_data['Curing_days']  # If you have an identifier column in your train data

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictionsAutoMLTrainIsoForest.csv', index=False)

print('Predictions saved to predictions.csv')

In [ ]:
import pandas as pd

# Assuming 'predictor' is your trained AutoGluon predictor and 'test_data' is your test dataset
# Get the predictions
predictions = predictor.predict(test_data.drop(columns=['CS']))  # Drop the target column for predictions

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['CS'])

# Optionally, include the actual values and any identifier columns
actuals = test_data['CS'].values
predictions_df['Actual_CS'] = actuals
predictions_df['CD'] = test_data['CD']  # If you have an identifier column in your test data

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictionsAutoMLTest42.csv', index=False)

print('Predictions saved to predictions.csv')

In [ ]:
import numpy as np

# Assuming 'predictor' is your trained AutoGluon predictor and 'test_data' is your test dataset
# Evaluate the model to get predictions
predictions = predictor.predict(test_data.drop(columns=['UCS']))  # Drop the target column for predictions
actuals = test_data['UCS'].values  # Get the actual values from the target column


# Calculate a20-index
def calculate_a20_index(actual, predicted):
    within_20_percent = np.abs((actual - predicted) / actual) <= 0.20
    m20 = np.sum(within_20_percent)
    M = len(actual)
    return (m20 / M) * 100

# Compute MAPE
mape = calculate_mape(actuals, predictions)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')

# Compute a20-index
a20_index = calculate_a20_index(actuals, predictions)
print(f'a20-index: {a20_index:.4f}%')


In [ ]:
pwd